In [1]:
import torch
import torchvision
import torchvision.models as models
from tqdm import tqdm
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset

In [2]:
original_dataset_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/train_images/'
path = original_dataset_dir

In [3]:
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [4]:
norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [37]:
class RetinopathyDatasetTrain(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/train_images/', self.data.loc[idx, 'id_code'] + '.png')
        try:
            image = Image.open(img_name)
            image = self.transform(image)
            return {'image': image}

SyntaxError: unexpected EOF while parsing (<ipython-input-37-fb2cab57d2b1>, line 14)

In [5]:
def getRadius(img):
#     print(type(img.shape))
    circles = cv2.HoughCircles(img,3,1,max(img.shape)/2,param1=50,param2=30,minRadius= int(max(img.shape)/6),maxRadius=max(img.shape))
    return circles[0][0][2]

In [33]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = RetinopathyDatasetTest(csv_file='/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/sample_submission.csv',
                                      transform=test_transform)

In [36]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=4)

for i, x_batch in enumerate(test_data_loader):
    x_batch = x_batch["image"]
    plt.imshow(x_batch)
    plt.show()
    break

TypeError: Traceback (most recent call last):
  File "/anaconda3/envs/learn-env/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/anaconda3/envs/learn-env/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-13-f3238a58883c>", line 12, in __getitem__
    image = self.transform(image)
  File "/anaconda3/envs/learn-env/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/anaconda3/envs/learn-env/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 196, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "/anaconda3/envs/learn-env/lib/python3.6/site-packages/torchvision/transforms/functional.py", line 229, in resize
    raise TypeError('img should be PIL Image. G

In [6]:
sm_path = path + '/sm'
imgs = []

In [7]:
smudge = []

In [12]:
for img in os.listdir(path):
    if img.endswith('.png'):
        img_array = cv2.imread(os.path.join(path,img))
        try:
            r = getRadius(img_array)
            crop_r = round(r*0.8) 
        except:
            crop_r = (img_array.shape[1]/2)*0.8
            
        center_x = int(img_array.shape[1]/2)
        center_y = int(img_array.shape[0]/2)
    
        left_border = int(center_x - crop_r)
        right_border = int(center_x + crop_r)
        upper_border = int(center_y - crop_r)
        lower_border = int(center_y + crop_r)
    
        if left_border < 0:
            difference = 0- left_border
            left_border = 0
            right_border -=difference
            upper_border += difference
            lower_border -= difference
        if upper_border < 0:
            difference = 0- upper_border
            upper_border = 0
            lower_border -= difference
            left_border += difference
            right_border -= difference

        cropped = img_array[upper_border:lower_border, left_border:right_border]
        a = cv2.resize(cropped, (224,224), interpolation = cv2.INTER_CUBIC)
#         lab = cv2.cvtColor(a, cv2.COLOR_BGR2LAB)
#         lab_planes = cv2.split(lab)
#         clahe = cv2.createCLAHE(clipLimit= 1,tileGridSize=(round(a.shape[0]/6),round(a.shape[1]/6)))
#         lab_planes[0] = clahe.apply(lab_planes[0])
#         lab = cv2.merge(lab_planes)
#         a = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
       
#         a =cv2.medianBlur(a,5)
#         a=cv2.addWeighted(a, 4, cv2.GaussianBlur(a,(0,0), 30), -4, 128)
        a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        a = torch.from_numpy(a)
        print(a.size())
        a = norm(a)
        imgs.append(a)

        plt.imshow(a)  # graph it
        plt.show()
        gc.collect()
        break
#         os.chdir(sm_path)
#         cv2.imwrite("sm"+"_"+img ,a)
#         os.chdir(path)   
#     print(ctr)

torch.Size([224, 224, 3])


RuntimeError: The size of tensor a (224) must match the size of tensor b (3) at non-singleton dimension 0

In [60]:
smudges = np.asarray(smudge)

In [61]:
smudges[0].shape

(512, 512, 3)

In [62]:
beep = [smudges[np.random.choice(smudges.shape[0],len(smudges)//10,replace = False)] for a in range (11)]

In [63]:
bleh = beep[0].reshape(beep[0].shape[0],beep[0].shape[1]*beep[0].shape[2]*beep[0].shape[3])

In [64]:
bleh_norm = bleh/255
b_cov = np.cov(bleh_norm, rowvar = True)
U, S, V = np.linalg.svd(b_cov)
epsilon = 0.1
b_ZCA = U.dot(np.diag(1.0/np.sqrt(S+epsilon))).dot(U.T).dot(bleh_norm)
bleh_ZCA_rescaled = (b_ZCA - b_ZCA.min())/(b_ZCA.max()-b_ZCA.min())

In [21]:
npimgs = np.asarray(imgs)

In [22]:
stacks = [npimgs[np.random.choice(npimgs.shape[0],len(npimgs)//10, replace = False)] for a in range(11)]

In [15]:
comps = []
epsilon = 0.1

for stack in stacks:
    X = stack.reshape(stack.shape[0], stack.shape[1]*stack.shape[2]*stack.shape[3])
    X_norm = X/255
    cov = np.cov(X_norm, rowvar=True)
    U,S,V = np.linalg.svd(cov)
    comps.append((U,S))


In [17]:
X_ZCA = U.dot(np.diag(1.0/np.sqrt(S + epsilon))).dot(U.T).dot(X_norm)


(366, 786432)


In [22]:
X_ZCA_rescaled = (X_ZCA - X_ZCA.min()) / (X_ZCA.max() - X_ZCA.min())

In [35]:
X_ZCA_rescaled.shape

(366, 786432)

In [29]:
for u in range(70,70+X_ZCA_rescaled.shape[0]//10):
    b= X_ZCA_rescaled[u].reshape(512,512,3)
    c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(c,100,200)
    plt.imshow(edges)
    plt.show()

error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<1, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::SizePolicy::NONE>::CvtHelper(InputArray, OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<1, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::SizePolicy::NONE]'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)


In [8]:
import gc; gc.enable()

In [2]:
gc.collect()

0

In [11]:
res_fifty = keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 24s 0us/step
